In [ ]:
import os
import glob
import subprocess
import time

import pandas as pd
import numpy as np
import json

import shutil

### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Install cmake and OpenPose dependencies

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Selecting previously unselected package libgflags2.2.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

### Initialize helper functions

In [ ]:
def convert_json2csv(json_directory, activity_folder):
    # determine the number of frames
    nframes = len(os.listdir(json_directory))

    # initialize res to be array of NaN
    res = np.zeros((nframes,75))
    res[:] = np.nan

    PROJ_JSON_ROOT = os.path.join('/content/openpose/output/')

    # read in JSON files
    for frame in range(0,nframes):
        
        test_image_json = os.path.join(PROJ_JSON_ROOT, f'{activity_folder}_{str(frame).zfill(12)}_keypoints.json')

        # if not os.path.isfile(test_image_json):
        #     break
        with open(test_image_json) as data_file:  
            data = json.load(data_file)

        for person in data['people']:
            keypoints = person['pose_keypoints_2d']
            xcoords = [keypoints[i] for i in range(len(keypoints)) if i % 3 == 0]
            counter = 0
            res[frame,:] = keypoints
            break

    return res

In [ ]:
PROJ_SRC_ROOT = os.path.join('/content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/sample_activity_videos/')
PROJ_SAVE_ROOT = os.path.join('/content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/sample_activity_outputs_openpose/')

sample_activity_videos = os.listdir(PROJ_SRC_ROOT)[-3:]

run_log_data = {}


for activity_type in sample_activity_videos:
    activity_lst = os.listdir(os.path.join(PROJ_SRC_ROOT, activity_type))

    print(activity_type)
    for activity_folder in activity_lst:
        raw_video_lst = os.listdir((os.path.join(PROJ_SRC_ROOT, activity_type, activity_folder)))

        print(f'\t{activity_folder:20}')
        for raw_video_file in raw_video_lst:
            print(f'\t\t{raw_video_file:20}')
#         src_folder_path = os.path.join(PROJ_SRC_ROOT, activity_type, raw_video_file)
#         activity_folder = raw_video_file

#         print(f'\t{activity_folder:20}')

# src_file_path

### Convert Local Video to Time Series

In [ ]:
PROJ_SRC_ROOT = os.path.join('/content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/sample_activity_videos/')
PROJ_SAVE_ROOT = os.path.join('/content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/sample_activity_outputs_openpose/')

sample_activity_videos = os.listdir(PROJ_SRC_ROOT)[-3:]

run_log_data = {}


for activity_type in sample_activity_videos:
    activity_lst = os.listdir(os.path.join(PROJ_SRC_ROOT, activity_type))

    print(activity_type)
    for activity_folder in activity_lst:
        raw_video_lst = os.listdir((os.path.join(PROJ_SRC_ROOT, activity_type, activity_folder)))

        print(f'\t{activity_folder:20}')
        for raw_video_file in raw_video_lst:
            
            src_file_path = os.path.join(PROJ_SRC_ROOT, activity_type, activity_folder, raw_video_file)
            raw_video_folder = raw_video_file[:-4]

            print(f'\t\t{raw_video_folder:20}')

            #Instantiate list for each activity
            run_log_data[activity_folder] = []
            
            #Append file size to run logs
            file_size = os.path.getsize(src_file_path)
            run_log_data[activity_folder].append(file_size)

            #Clear json files from previous run
            for f in glob.glob('/content/openpose/output/*.json'):
                os.remove(f)

            #Clear avi files from previous run 
            subprocess.run(["rm", "openpose.avi"])

            #Run Pose Detection
            start = time.time()
            os.chdir('/content/openpose')
            subprocess.run(["./build/examples/openpose/openpose.bin", 
                    "--video", src_file_path, 
                    "--write_json", "./output/", 
                    "--display", "0",  
                    "--write_video", "../openpose.avi"], 
                    shell=False)
            end = time.time()
            pose_detection_time = round(end-start, 1)
            run_log_data[activity_folder].append(pose_detection_time)  

            print(f'\t\t Pose Detected in {pose_detection_time:5}s')

            #AVI to MP4 conversion
            start = time.time()
            os.chdir('/content')
            subprocess.run(["ffmpeg", "-y", "-loglevel", "info", "-i", "openpose.avi", "output.mp4"], shell=False)
            end = time.time()
            video_convert_time = round(end-start, 1)
            run_log_data[activity_folder].append(video_convert_time) 

            print(f'\t\t Video Converted in {video_convert_time:5}s') 


            #Create csv output from json files
            biomech_df = pd.DataFrame(convert_json2csv("/content/openpose/output/", raw_video_folder))


            #########################################  SAVING  ##########################################

            # Make specific folder for activity files
            path = os.path.join(PROJ_SAVE_ROOT, activity_type, activity_folder, raw_video_folder)

            if os.path.exists(path):
                pass
            else:
                os.mkdir(path)

            #### DATA
            data_save_path = os.path.join(PROJ_SAVE_ROOT, activity_type, activity_folder, raw_video_folder, f"{raw_video_folder}.csv")
            biomech_df.to_csv(data_save_path)

            print(f'\t\t Save path : {data_save_path:20}')

            #### VIDEOS
            avi_video_src_file_path = '/content/openpose.avi'
            mp4_video_src_file_path = '/content/output.mp4'

            avi_save_path = os.path.join(PROJ_SAVE_ROOT, activity_type, activity_folder, raw_video_folder, f"{raw_video_folder}.avi")
            mp4_save_path = os.path.join(PROJ_SAVE_ROOT, activity_type, activity_folder, raw_video_folder, f"{raw_video_folder}.mp4")

            shutil.move(avi_video_src_file_path, avi_save_path)
            shutil.move(mp4_video_src_file_path, mp4_save_path)

            print(f'\t\t Video Saved') 


05_running_drills
	b-skips             
		b-skips_01          
		 Pose Detected in  11.5s
		 Video Converted in  10.8s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/sample_activity_outputs_openpose/05_running_drills/b-skips/b-skips_01/b-skips_01.csv
		 Video Saved
		b-skips_02          
		 Pose Detected in  13.0s
		 Video Converted in  10.4s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/sample_activity_outputs_openpose/05_running_drills/b-skips/b-skips_02/b-skips_02.csv
		 Video Saved
		b-skips_03          
		 Pose Detected in   9.6s
		 Video Converted in   8.4s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/sample_activity_outputs_openpose/05_running_drills/b-skips/b-skips_03/b-skips_03.csv
		 Video Saved
		b-skips_04          
		 Pose Detected in   9.7s
		 Video Converted in   6.7s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/sample_activity_outputs_openpose/05_running_

In [ ]:
biomech_df = pd.DataFrame.from_dict(run_log_data, orient='index', columns=['File_Size', "Pose_Detection_Time(s)", "Video_Convert_Time(s)"])


log_save_path = '/content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/OpenPose_sample_video_logs2.csv'
biomech_df.to_csv(log_save_path, index=True)

biomech_df

,File_Size,Pose_Detection_Time(s),Video_Convert_Time(s)
b-skips,10233404,17.1,20.6
leg_sweeps,691860,7.5,4.7
a-skips,458826,7.8,2.8
c-skips,551298,9.2,4.2
burpees,29627561,44.6,62.0
squats,16464636,29.6,44.4
jumping_jacks,5167398,40.1,32.4
mountain_climbers,2378744,30.4,17.9
pushups,6257778,49.9,40.3
defensive_line,3122010,11.9,11.8
